# Testing with Databases

### Introduction

Testing applications that use databases is kind of tricky.  For example, if we want to test that a function properly can determine the number of premium users that we have, this means we need to populate a database with premium users, and then ensure our function selects them.  But should we really add new records to our database, just for the purposes of tests?  

In this lesson, we'll learn how to test functions that require updating our database and how Pytest can help us in accomplishing this task.

### A testing strategy

To see why writing tests for a database is tricky, let's stick with our `premium_user_count` function.  The function should find all of the users in our users that are premium.  It looks like the following:

```python
# index.py
def premium_user_count():
    cursor.execute("SELECT COUNT(*) FROM users WHERE category = 'premium'")
    return cursor.fetchall()
```

And, if there is one premium user in our database, then our test looks like the following:

```python
# test_index.py
from index import *
def test_premium_user_count():
    assert premium_user_count() == 1
```

So notice that above test will only work properly if we first add exactly one premium users to our database - no more, no less.  So we can see that writing tests for methods that rely on a database is tricky because we first need to ensure that our database is in a exactly the correct state.  

### How we do it

Ensuring proper setup of our database requires a couple of things.  First, it means that we will create a database just for running our tests.  This way, we can add and remove records to the our test database, and it will not affect any real data.  

In addition to creating a test database, we'll create certain records just for the purpose of running a test, and then after running the test we'll destroy the records.  This way we can be confident about what is and is not in our database.  

So now in our test above, we can do something like the following:

In [1]:
def test_premium_user_count():
    # setup db with premium customer (and non-premium)
    
    insert_into = 'INSERT INTO customers (first_name, last_name, category) VALUES (%s, %s)'
    cursor.execute(insert_into, 'bob', 'smith', 'premium')
    cursor.execute(insert_into, 'fred', 'samuel', 'standard')
    cursor.commit()
    # run test
    assert premium_user_count() == 1
    
    # clean up database
    cursor.execute('DELETE FROM customers;')
    cursor.commit()

So in the above we first ensure our releveant table is clean, then we insert three premium records to the customers table.  Then we check that our `premium_user_count` function returns the correct number of customers, and finally, we clean up our database.

Of course, this looks pretty messy, and perhaps time consuming.  But don't worry, we'll clean up this code below.

### Pytest Fixtures

Pytest fixtures write us to place our setup of the database, and subsequent cleanup into a separate function -- our fixture.  So if we take another look at the messy code in our test from above:

In [1]:
# OLD MESSY VERSION
def test_premium_user_count():
    # setup db with premium customer (and non-premium)
    
    insert_into = 'INSERT INTO customers (first_name, last_name, category) VALUES (%s, %s)'
    cursor.execute(insert_into, 'bob', 'smith', 'premium')
    cursor.execute(insert_into, 'fred', 'samuel', 'standard')
    cursor.commit()
    # run test
    assert premium_user_count() == 1
    
    # clean up database
    cursor.execute('DELETE FROM customers;')
    cursor.commit()

We'll now move the inserting and deleting from our database to a separate function called a fixture.

In [3]:
import pytest
@pytest.fixture()
def db_with_premium_customers():
    # setup db with premium customer and non-premium 
    insert_into = 'INSERT INTO customers (first_name, last_name, category) VALUES (%s, %s)'
    cursor.execute(insert_into, 'bob', 'smith', 'premium')
    cursor.execute(insert_into, 'fred', 'samuel', 'standard')
    cursor.commit()
    yield
    # cleanup
    cursor.execute('DELETE FROM customers;')
    cursor.commit()

And now our test looks like the following.

In [5]:
def test_premium_user_count(db_with_premium_customers):   
    assert premium_user_count() == 1

So we can see that by passing `db_with_premium_customers` as an argument to our `test_premium_user_count` function, the test first calls our fixture.  This fixture then begins to run, and inserts the premium and standard customer into the database.  Then the `yield` keyword is reached.  This `yields` to the `test_premium_user_count` function, which then checks that the function returns 1, and then our fixture continues to run -- here cleaning up our customers table.

So that's a fixture with pytest.  They help to isolate the setup and teardown of our data.  And a benefit of them, is that we can reuse our fixture simply by passing the fixture to another test function.

The key to understanding fixtures is that they generally follow the same pattern:

```python
@pytest.fixture()
def db_fixture():
    # setup
    yield
    # teardown
```

We state that we are creating a fixture with the `@pytest.fixture()` decorator.  Then inside our fixture we first perform the setup -- here inserting records to the database -- then the `yield` keyword calls our test, and then our setup is "torn down" (by say, deleting the database records).

We get our test to call our fixture by just passing through the name of the fixture as an argument to the test: `test_function(db_fixture)`.

> Let's see the whole thing again:

In [3]:
import pytest
@pytest.fixture()
def db_with_premium_customers():
    # setup db with premium customer and non-premium 
    insert_into = 'INSERT INTO customers (first_name, last_name, category) VALUES (%s, %s)'
    cursor.execute(insert_into, 'bob', 'smith', 'premium')
    cursor.execute(insert_into, 'fred', 'samuel', 'standard')
    cursor.commit()
    yield
    # cleanup
    cursor.execute('DELETE FROM customers;')
    cursor.commit()
    
def test_premium_user_count(db_with_premium_customers):   
    assert premium_user_count() == 1

### Summary

In this lesson, we learned how to test database methods.  We do so by first creating a database just for our tests.  Then we use fixtures to both setup our database in the correct state, and cleanup our database when we are done.  

A fixture generally follows the following structure:

```python
@pytest.fixture()
def db_fixture():
    # setup
    yield
    # teardown
    
def test_function(db_fixture):
    pass
```

And in our scenario we can read these steps as the following:

* Setup: We first get our database in the correct state by adding some data.  
* Yield: Then we yield to our test, which is run (Does our function perform the query properly?)
* Teardown: Then we continue on with our fixture cleaning up our database by deleting all records